# Mate detection

## Basic Setup

In [4]:
# Clone mate-detection repo
!git clone https://github.com/75-70-Robots/mate-detection.git

Clonando en 'mate-detection'...
remote: Enumerating objects: 945, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 945 (delta 133), reused 150 (delta 133), pack-reused 795
Recibiendo objetos: 100% (945/945), 125.87 MiB | 7.09 MiB/s, listo.
Resolviendo deltas: 100% (522/522), listo.


In [64]:
# Clone yolov5 repo
!git clone https://github.com/ultralytics/yolov5

Clonando en 'yolov5'...
remote: Enumerating objects: 6903, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 6903 (delta 1), reused 2 (delta 0), pack-reused 6894
Recibiendo objetos: 100% (6903/6903), 8.78 MiB | 4.52 MiB/s, listo.
Resolviendo deltas: 100% (4740/4740), listo.


In [11]:
# Clone modified yolov5 repo for second stage classifier
!git clone https://github.com/necla-ml/yolov5.git

Clonando en 'yolov5'...
remote: Enumerating objects: 4237, done.
remote: Total 4237 (delta 0), reused 0 (delta 0), pack-reused 4237
Recibiendo objetos: 100% (4237/4237), 7.91 MiB | 3.60 MiB/s, listo.
Resolviendo deltas: 100% (2836/2836), listo.


In [6]:
!pip3 install -r yolov5/requirements.txt

In [46]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 121 kB 1.3 MB/s eta 0:00:01


     |████████████████████████████████| 243 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 10.6 MB/s eta 0:00:01


In [1]:
import os
import cv2
import copy
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import shutil
import yaml
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

## Data parsing

In [2]:
IMG_DIR = "mate-detection/images/mates/"
XML_DIR = "mate-detection/labels/mates/"
labels = ["mate"]


def leer_mate_labels(ann_dir, img_dir, labels=[]):
    all_imgs = []
    for ann in sorted(os.listdir(ann_dir)):
        img = {'object':[]}
        tree = ET.parse(ann_dir + ann)
        
        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = img_dir + elem.text
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
                
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text.lower()
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                            
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text)))
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text)))
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text)))
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text)))

        if len(img['object']) > 0:
            all_imgs += [img]
                        
    return all_imgs

train_imgs = leer_mate_labels(XML_DIR, IMG_DIR, labels)
print(len(train_imgs))
print(train_imgs)

361
[{'object': [{'name': 'mate', 'xmin': 246, 'ymin': 338, 'xmax': 717, 'ymax': 951}], 'filename': 'mate-detection/images/mates/Manes-01.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 192, 'ymin': 450, 'xmax': 628, 'ymax': 887}], 'filename': 'mate-detection/images/mates/Manes-02.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 280, 'ymin': 515, 'xmax': 733, 'ymax': 1114}], 'filename': 'mate-detection/images/mates/Manes-03.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 200, 'ymin': 483, 'xmax': 540, 'ymax': 955}], 'filename': 'mate-detection/images/mates/Manes-04.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 86, 'ymin': 410, 'xmax': 586, 'ymax': 875}], 'filename': 'mate-detection/images/mates/Manes-05.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 132, 'ymin': 557, 'xmax': 539, 'ymax': 991}], 'filename': 'mate-detection/images/mates/Manes-06.jpeg', 

In [3]:
def create_labels_id(labels):
    labels_id = {}
    id_count = 0
    for label in labels:
        labels_id[label] = id_count
        id_count += 1
    return labels_id

labels_id = create_labels_id(labels)
labels_id

{'mate': 0}

In [4]:
def split_dataset(data, train_percentage):
    train_valid_split = int(train_percentage*len(data))
    np.random.shuffle(data)
    validation_imgs = data[train_valid_split:]
    training_imgs = data[:train_valid_split]
    return training_imgs, validation_imgs
training_imgs, validation_imgs = split_dataset(train_imgs, 0.8)
print('train:',len(training_imgs), 'validate:',len(validation_imgs))

train: 288 validate: 73


In [5]:
def transform_dict_in_dataframe(img_dict):
    df = pd.DataFrame()
    for image in img_dict:
        for obj in image['object']:
            x1 = obj['xmin']
            x2 = obj['xmax']
            y1 = obj['ymin']
            y2 = obj['ymax']
            w = image['width']
            h = height = image['height']
            width = (x2-x1)/w
            height = (y2-y1)/h
            xcenter = (x1+(x2-x1)/2)/w
            ycenter = (y1+(y2-y1)/2)/h
            label = obj['name']
            filename = image['filename']
            new_row = {'filename': filename, 'label': label, 'xcenter': xcenter, 'ycenter': ycenter, 'width': width,
                      'height': height, 'image_height':h, 'image_width':w}
            df = df.append(new_row, ignore_index=True)

    return df

In [6]:
def create_yolo_dataset(train_dataframe, labels_id, trained_img_path, trained_lbl_path):
    
    if os.path.exists(trained_img_path):
        shutil.rmtree(trained_img_path)
        
    if os.path.exists(trained_lbl_path):
        shutil.rmtree(trained_lbl_path)
        
    os.makedirs(trained_img_path)
    os.makedirs(trained_lbl_path)

    img_names = list(train_dataframe['filename'].unique())

    for full_name in img_names:
        name = full_name.split('/')[-1]
        if not os.path.exists(trained_img_path+name):
            shutil.copy(IMG_DIR+name,trained_img_path+name)
        img = cv2.imread(IMG_DIR+name)
        h, w = img.shape[:2]
        f_name = name.split('.')[0]

        with open(trained_lbl_path+f_name+".txt",'w') as label:
            ldf=train_dataframe.loc[train_dataframe['filename']==full_name]
            for index, row in ldf.iterrows():
                l = labels_id[row['label']]
                xcenter = row['xcenter']
                ycenter = row['ycenter']
                width = row['width']
                height = row['height']
                l_data = f"{l} {xcenter} {ycenter} {width} {height}\n"
                label.write(l_data)

In [7]:
def create_yolo_yaml_configuration(training_img_path, validation_img_path, labels, yaml_name, yolo_model='s'):
    yaml_data = {
      'train': training_img_path,
      'val': validation_img_path,
      'nc':len(labels),
      'names':labels
    }

    with open(yaml_name,'w') as f:
        yaml.dump(yaml_data, f)

    with open(f'yolov5/models/yolov5{yolo_model}.yaml') as f:
        ydata = yaml.load(f, Loader=yaml.FullLoader) 

    ydata['nc'] = len(labels)
    with open('yolov5_custom.yaml','w') as cy:
        yaml.dump(ydata,cy)

In [8]:
TRAINED_IMG_PATH = 'data/images/'
TRAINED_LBL_PATH = 'data/labels/'
train_df = transform_dict_in_dataframe(training_imgs)
create_yolo_dataset(train_df, labels_id, TRAINED_IMG_PATH, TRAINED_LBL_PATH)
train_df

,filename,height,image_height,image_width,label,width,xcenter,ycenter
0,mate-detection/images/mates/ffusaro-07.jpeg,0.482812,1280.0,960.0,mate,0.451042,0.433854,0.242188
1,mate-detection/images/mates/lolguin-21.jpeg,0.360938,640.0,384.0,mate,0.572917,0.421875,0.682031
2,mate-detection/images/mates/Manes-24.jpeg,0.557813,1280.0,720.0,mate,0.681944,0.504861,0.492188
3,mate-detection/images/mates/cflorez-03.jpeg,0.396875,1280.0,720.0,mate,0.709722,0.620139,0.543750
4,mate-detection/images/mates/aaackermann-31.jpg,0.310156,1280.0,958.0,mate,0.540710,0.625261,0.336328
...,...,...,...,...,...,...,...,...
283,mate-detection/images/mates/apayaslian-16.jpg,0.488889,2160.0,2160.0,mate,0.376389,0.462731,0.325000
284,mate-detection/images/mates/fmazzoni-08.jpeg,0.347545,774.0,1032.0,mate,0.247093,0.476260,0.464470
285,mate-detection/images/mates/cflorez-21.jpeg,0.347656,1280.0,720.0,mate,0.445833,0.439583,0.798047
286,mate-detection/images/mates/fmazzoni-28.jpeg,0.458656,774.0,1032.0,mate,0.378876,0.352229,0.424419


In [9]:
VALIDATION_IMG_PATH = 'valid/images/'
VALIDATION_LBL_PATH = 'valid/labels/'
valid_df = transform_dict_in_dataframe(validation_imgs)
create_yolo_dataset(valid_df, labels_id, VALIDATION_IMG_PATH, VALIDATION_LBL_PATH)
valid_df

,filename,height,image_height,image_width,label,width,xcenter,ycenter
0,mate-detection/images/mates/fmazzoni-22.jpeg,0.337209,774.0,1032.0,mate,0.241279,0.537306,0.445090
1,mate-detection/images/mates/npfernandeztheille...,0.662562,406.0,600.0,mate,0.338333,0.684167,0.668719
2,mate-detection/images/mates/apayaslian-35.jpg,0.493519,2160.0,2160.0,mate,0.387963,0.499074,0.334722
3,mate-detection/images/mates/fmazzoni-16.jpeg,0.405039,1032.0,774.0,mate,0.337209,0.592377,0.442829
4,mate-detection/images/mates/apayaslian-04.jpg,0.407407,2160.0,2160.0,mate,0.380556,0.475000,0.404630
...,...,...,...,...,...,...,...,...
69,mate-detection/images/mates/aaackermann-30.jpg,0.214844,1280.0,958.0,mate,0.577244,0.336639,0.579297
70,mate-detection/images/mates/fmazzoni-35.jpeg,0.312016,1032.0,774.0,mate,0.397933,0.416021,0.588178
71,mate-detection/images/mates/fmazzoni-19.jpeg,0.370801,774.0,1032.0,mate,0.169574,0.658430,0.451550
72,mate-detection/images/mates/cflorez-25.jpeg,0.250781,1280.0,720.0,mate,0.351389,0.503472,0.512109


In [10]:
YAML_NAME = 'mate.yaml'
yolo_model = 'x'
create_yolo_yaml_configuration(TRAINED_IMG_PATH,VALIDATION_IMG_PATH,labels,YAML_NAME,yolo_model)

## Train

In [12]:
!python3 yolov5/train.py --img 640 --batch 1 --epochs 200 --data mate.yaml --weights yolov5x.pt --name yolov5_mates --cfg yolov5_custom.yaml

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-145-ga1c3572 torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Namespace(adam=False, artifact_alias='latest', batch_size=1, bbox_interval=-1, bucket='', cache_images=False, cfg='yolov5_custom.yaml', data='mate.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5_mates', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5_mates40', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=1, upload_dataset=False, weights='yolov5x.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-06 23:25:44.914527: W te


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     8/199     2.47G   0.04152   0.01881         0   0.06033         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.362       0.493       0.319      0.0952

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     9/199     2.47G   0.03821   0.01849         0    0.0567         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.312       0.409       0.239      0.0444

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    10/199     2.47G   0.03651   0.01893         0   0.05543         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.735       0.534       0.603  

               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.716       0.726       0.735       0.464

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    57/199     2.47G    0.0153   0.01126         0   0.02656         4       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.783        0.74       0.775       0.461

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    58/199     2.47G   0.01758   0.01167         0   0.02925         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73         0.8       0.821        0.84       0.502

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    59/199     2.47G   0.01599   0.01184         0   0.02783         3


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    81/199     2.47G    0.0134  0.009807         0   0.02321         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.773       0.795        0.81       0.497

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    82/199     2.47G   0.01402  0.009469         0   0.02349         1       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.859       0.753       0.847       0.584

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    83/199     2.47G   0.01371   0.01001         0   0.02373         4       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.855       0.808       0.862  

               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.811       0.822       0.796       0.529

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   130/199     2.47G   0.01009  0.006887         0   0.01697         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.833       0.822       0.794       0.531

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   131/199     2.47G   0.00947  0.007153         0   0.01662         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.782       0.836       0.793       0.522

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   132/199     2.47G  0.009542  0.007564         0   0.01711         4


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   154/199     2.47G   0.00864  0.006799         0   0.01544         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.753       0.836       0.826       0.524

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   155/199     2.47G  0.008139  0.006869         0   0.01501         3       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.765       0.849        0.82       0.524

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   156/199     2.47G  0.008573   0.00632         0   0.01489         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all          73          73       0.753       0.795       0.805  

## Material recognition

In [11]:
material_labels = ['madera', 'plastico', 'calabaza', 'metal']
material_imgs = leer_mate_labels(XML_DIR, IMG_DIR, material_labels)
len(material_imgs)

347

In [12]:
train_materials, test_materials = split_dataset(material_imgs, 0.8)

In [13]:
def create_images_directory(base_dir, labels):
    for label in labels:
        full_path = f"{base_dir}{label}/"
        if os.path.exists(full_path):
            shutil.rmtree(full_path)
        os.makedirs(full_path)

In [14]:
def crop_images(train_images, path_to_save_cropped_image):
    new_trained_image = []
    for image in train_images:
        x_min = image['object'][0]['xmin']
        x_max = image['object'][0]['xmax']
        y_min = image['object'][0]['ymin']
        y_max = image['object'][0]['ymax']
        img = cv2.imread(image['filename'])
        crop_img = img[y_min:y_max, x_min:x_max]
        image_name = image['filename'].split('/')[-1]
        image_class = image['object'][0]['name']
        full_path_name = f"{path_to_save_cropped_image}{image_class}/{image_name}"
        cv2.imwrite(full_path_name,crop_img)
        aux_image = image
        aux_image['filename'] = full_path_name
        new_trained_image.append(aux_image)
        
    return new_trained_image

In [15]:
MATERIAL_TRAINED_IMG_PATH = 'material_data/train/'
MATERIAL_VALIDATING_IMG_PATH = 'material_data/valid/'
create_images_directory(MATERIAL_TRAINED_IMG_PATH, material_labels)
create_images_directory(MATERIAL_VALIDATING_IMG_PATH, material_labels)
training_material_images = crop_images(train_materials, MATERIAL_TRAINED_IMG_PATH)
validation_material_images = crop_images(test_materials, MATERIAL_VALIDATING_IMG_PATH)

In [16]:
trainging_material_df = transform_dict_in_dataframe(training_material_images)
trainging_material_df = trainging_material_df[['filename','label']]
trainging_material_df

,filename,label
0,material_data/train/plastico/lolguin-19.jpeg,plastico
1,material_data/train/metal/apayaslian-13.jpg,metal
2,material_data/train/plastico/aaackermann-03.jpg,plastico
3,material_data/train/madera/apayaslian-25.jpg,madera
4,material_data/train/madera/Manes-01.jpeg,madera
...,...,...
272,material_data/train/calabaza/fmazzoni-06.jpeg,calabaza
273,material_data/train/plastico/aaackermann-20.jpg,plastico
274,material_data/train/plastico/aaackermann-19.jpg,plastico
275,material_data/train/metal/fmazzoni-25.jpeg,metal


In [17]:
validation_material_df = transform_dict_in_dataframe(validation_material_images)
validation_material_df = validation_material_df[['filename','label']]
validation_material_df

,filename,label
0,material_data/valid/metal/lolguin-40.jpeg,metal
1,material_data/valid/calabaza/lolguin-22.jpeg,calabaza
2,material_data/valid/plastico/cflorez-22.jpeg,plastico
3,material_data/valid/plastico/cflorez-09.jpeg,plastico
4,material_data/valid/metal/cflorez-02.jpeg,metal
...,...,...
65,material_data/valid/madera/ffusaro-12.jpg,madera
66,material_data/valid/calabaza/fmazzoni-18.jpeg,calabaza
67,material_data/valid/plastico/lolguin-18.jpeg,plastico
68,material_data/valid/calabaza/fmazzoni-30.jpeg,calabaza


## Train image classifier

In [18]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [19]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'material_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [20]:
image_datasets['train'].class_to_idx

{'calabaza': 0, 'madera': 1, 'metal': 2, 'plastico': 3}

In [21]:
device = torch.device("cpu")

In [22]:
model_ft = models.resnet101(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(material_labels))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------
train Loss: 1.4433 Acc: 0.3430
valid Loss: 1.1721 Acc: 0.5429

Epoch 1/199
----------


In [39]:
PATH = 'weights/resnet101.pt'
torch.save(model_ft, PATH)

## Detection

### Image

In [30]:
!python3 modified_yolov5/detect.py --source test_images/ --weights runs/train/yolov5_mates38/weights/best.pt --conf 0.6 --second-classifier

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, second_classifier=True, source='test_images/', update=False, view_img=False, weights=['runs/train/yolov5_mates38/weights/best.pt'])
YOLOv5  torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Fusing layers... 
Model Summary: 476 layers, 87198694 parameters, 0 gradients, 217.1 GFLOPS
['mate']
['calabaza', 'madera', 'metal', 'plastico']
image 1/31 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate-calabaza-00.jpg: 384x640 Done. (0.286s)
image 2/31 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate-calabaza-01.jpg: 640x448 Done. (0.162s)
image 3/31 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate-calabaza-04-cuero.jpg: 640x544 Done. (0.199s)
image 4/31 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate-calabaza-06.jpg: 640x544 Done. (0.199s)
image 5/31 /home/pay

### Video

### Webcam

In [33]:
!python3 modified_yolov5/detect.py --source 0 --weights runs/train/yolov5_mates38/weights/best.pt --img 640 --conf 0.4 --second-classifier

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, second_classifier=True, source='0', update=False, view_img=False, weights=['runs/train/yolov5_mates38/weights/best.pt'])
YOLOv5  torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Fusing layers... 
Model Summary: 476 layers, 87198694 parameters, 0 gradients, 217.1 GFLOPS
['mate']
['calabaza', 'madera', 'metal', 'plastico']
1/1: 0...  success (640x480 at 30.00 FPS).

0: 480x640 Done. (1.025s)
0: 480x640 Done. (0.186s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.177s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.176s)
0: 480x64